In [39]:
!pip install yfinance
!pip install psycopg2-binary
!pip install python-dotenv
import pandas as pd

In [40]:
import yfinance as yf
hmy = yf.Ticker('HMY')
hist = hmy.history(period="5y")
hist['Date']=hist.index
hist=hist.reset_index(drop=True)
hist= hist.rename(columns={"Open":"Apertura","High":"Precio_alto","Low":"Precio_bajo","Close":"Cierre","Volume":"Volumen","Dividends":"Dividendos","Stock Splits":"Stock","Date":"Dia"})
hist= hist.drop(columns=["Dividendos","Stock","Volumen"])
hist

,Apertura,Precio_alto,Precio_bajo,Cierre,Dia
0,1.529894,1.539577,1.491162,1.510528,2018-06-18 00:00:00-04:00
1,1.529894,1.558942,1.520211,1.549259,2018-06-19 00:00:00-04:00
2,1.558942,1.597674,1.539577,1.539577,2018-06-20 00:00:00-04:00
3,1.549260,1.626723,1.520211,1.578308,2018-06-21 00:00:00-04:00
4,1.578308,1.587991,1.549259,1.549259,2018-06-22 00:00:00-04:00
...,...,...,...,...,...
1254,4.620000,4.660000,4.540000,4.660000,2023-06-12 00:00:00-04:00
1255,4.660000,4.700000,4.460000,4.480000,2023-06-13 00:00:00-04:00
1256,4.600000,4.660000,4.490000,4.530000,2023-06-14 00:00:00-04:00
1257,4.560000,4.590000,4.500000,4.560000,2023-06-15 00:00:00-04:00


In [41]:
hist.duplicated()

0       False
1       False
2       False
3       False
4       False
        ...  
1254    False
1255    False
1256    False
1257    False
1258    False
Length: 1259, dtype: bool

In [42]:
Harmony_sin_duplicados= hist.drop_duplicates("Apertura")
Harmony_sin_duplicados

,Apertura,Precio_alto,Precio_bajo,Cierre,Dia
0,1.529894,1.539577,1.491162,1.510528,2018-06-18 00:00:00-04:00
1,1.529894,1.558942,1.520211,1.549259,2018-06-19 00:00:00-04:00
2,1.558942,1.597674,1.539577,1.539577,2018-06-20 00:00:00-04:00
3,1.549260,1.626723,1.520211,1.578308,2018-06-21 00:00:00-04:00
4,1.578308,1.587991,1.549259,1.549259,2018-06-22 00:00:00-04:00
...,...,...,...,...,...
1248,4.750000,4.800000,4.600000,4.620000,2023-06-02 00:00:00-04:00
1251,4.650000,4.810000,4.560000,4.560000,2023-06-07 00:00:00-04:00
1252,4.640000,4.710000,4.560000,4.600000,2023-06-08 00:00:00-04:00
1255,4.660000,4.700000,4.460000,4.480000,2023-06-13 00:00:00-04:00


In [43]:
Harmony_sin_duplicados.describe()

,Apertura,Precio_alto,Precio_bajo,Cierre
count,1159.000000,1159.000000,1159.000000,1159.000000
mean,3.453514,3.530570,3.373097,3.450067
std,1.179054,1.197406,1.148151,1.172031
min,1.394334,1.481479,1.384651,1.394334
25%,2.701521,2.794175,2.622188,2.699347
50%,3.466468,3.543931,3.398688,3.473134
75%,4.199316,4.279691,4.105538,4.183001
max,7.213739,7.368666,6.787693,6.874839


In [44]:
import os
import psycopg2
env = os.environ

In [45]:
try:
    conn =psycopg2.connect( host=env['AWS_REDSHIFT_HOST'],
                           dbname="data-engineer-database",
                           user= env['AWS_REDSHIFT_USER'],
                           password= env['AWS_REDSHIFT_PASSWORD'],
                           port="5439"
                          )
    print("me conecte")
except Exception as e:
    print("odio esto")
    print(e)  

me conecte


In [46]:
cur= conn.cursor()
cur.execute ("SELECT * FROM Harmony_sin_duplicados")
results = cur.fetchall()
results

[]

In [47]:
from psycopg2.extras import execute_values
cur = conn.cursor ()
table_name="Harmony_sin_duplicados"
columns=["Apertura","precio_alto","Precio_bajo","Cierre","Dia"]
values=[tuple(x) for x in Harmony_sin_duplicados.to_numpy()]
insert_sql= f"INSERT INTO {table_name} ({','.join(columns)}) VALUES %s"
cur.execute("BEGIN")
execute_values(cur, insert_sql, values)
cur.execute("COMMIT")